# Jupyter Notebook shortcuts

* Esc: go to command mode
* Enter: go to edit mode


* m: switch cell to markup
* y: switch cell to code


* Shift-Enter: Execute cell
* dd: delete cell


* a: Insert new cell above
* b: Insert new cell below


* k: Move to cell above
* j: Move to cell below

In [ ]:
# this is a code cell

this is a markdown cell

---

# Follow along

## Supervised Learning - Typischer Ablauf

1. Fragestellung definieren
1. Metrik festlegen
1. Daten beschaffen
1. Explorative Datenanalyse
1. Daten aufbereiten
 1. Tidy Data
 1. Fehler bereinigen
 1. Null-Werte behandeln
 1. Ausreisser behandeln
 1. Kategorische Variablen behandeln
1. Feature Engineering
 1. Feature Extraction
 1. Feature Construction
 1. Feature Selection
1. Validierungsmethodik festlegen
1. Training (Model building)
 1. Algorithmen trainieren
 1. Hyperparameter optimieren
 1. Validieren
1. Ensembling
1. Prediction

## Fragestellung

* Welchen Preis erziehlt meine Immobilie?
* Ziel: Voraussage des Verkaufspreises einer Immobilie

## Metrik

* Ziel:
  * Möglichst geringe Abweichung zwischen wirklichem und vorausgesagtem Preis
  * Abweichungen gegen oben oder unten sind "gleich schlecht", d.h. es ist egal, ob wir uns gegeb oben oder gegen unten verschätzen, nur auf den Betrag der Abweichung kommt es an.
* Gewählte Metrik:
  * Root Mean Squared Error: $\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}$

### Theorie-Einschub

#### Bias-Variance Tradeoff

Beim Trainieren von Supervised Learning Algorithmen gibt es zwei Quellen von Fehlern, die dem erfolgreichen Generalisieren über das Trainingsset hinweg entgegenwirken:

* Bias: Unterschied zwischen effektiven (gemessenen) Wert und vom Algorithmus vorhergesagten Wert.
* Varianz: Sensitivität gegenüber Noise im Trainingsset

Wenn das eine sinkt, steigt normalerweise das andere. Das Ziel ist immer, beide zusammen zu optimieren.

#### Overfitting

* Hohe Varianz, tiefer Bias

#### Underfitting

* Hoher Bias, tiefe Varianz

## Daten beschaffen

Für dieses Beispiel verwenden wir das *Ames Housing Dataset*, welches Immobilieneigenschaften und Verkauspreise von Häusern aus Iowa beinhaltet.
* Paper: http://www.amstat.org/publications/jse/v19n3/decock.pdf
* Daten: https://ww2.amstat.org/publications/jse/v19n3/decock/AmesHousing.txt
* Codebook: https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline

In [ ]:
!head -n 3 AmesHousing.txt

In [ ]:
df = pd.read_csv("AmesHousing.txt", delimiter='\t')

In [ ]:
# nur damit Beispiel mit Dummies weiter unten anschaulicher wird
df = df.sample(frac=1, random_state=16).reset_index(drop=True)

Um das Beispiel etwas übersichtlicher zu gestalten, behalten wir nur einige wenige Spalten

In [ ]:
keep_columns = ['Lot Area', 'Bldg Type', 'Overall Qual', 'Overall Cond',
                'Year Built', 'Year Remod/Add', 'TotRms AbvGrd', 'Garage Area', 'SalePrice']
df = df[keep_columns]

## Explorative Datenanalyse

* Statistische Verteilung der Daten
* Class imbalance der Targetvariable
* Plots, Histogramme, Summary Statistics
* Korrelationen der Features untereinander und zum Target

In [ ]:
df.plot(kind='box', subplots=True, sharex=False, layout=(2,4), figsize=(18,8))

In [ ]:
df.plot(kind='kde', subplots=True, sharex=False, layout=(2,4), figsize=(18,8))

## Daten Aufbereiten

### Tidy Data Principles

1. Eine Variable pro Spalte (1NF)
1. Eine Beobachtung pro Zeile
1. Einen header mit verständlichen Bezeichnern
1. Eine Tabelle pro Art der Beobachtung
1. Eine Methode zum joinen von Tabellen

Punkte 1, 2 sind bereits erfüllt, 4 und 5 nicht relevant, da wir nur eine Tabelle haben. Wir passen lediglich noch die Spaltennamen an, um sie etwas verständlicher zu machen.

In [ ]:
df.columns = ['grdstkfläche (k)', 'haustyp (n)', 'qualität (o)', 'zustand (o)', 'baujahr (d)',
             'renovationsjahr (d)', 'zimmer (d)', 'garagenfläche (k)', 'PREIS']
df.head()

### Fehler bereinigen

* Tippfehler
* Unterschiedliche Schreibweisen
* Weitere Fehler

### Null-Werte behandeln

* Gründe für Null-Werte herausfinden
 * Programmierfehler
 * Messfehler, korrupter Wert
 * Techn. Problem (z.B. Sensorausfall)
 * Nichtmessbarkeit (z.B. Wert bei entsprechendem Element nicht relevant)
 * Null-Wert bedeutet 0, False oder Absenz
 
 
* Null-Werte zuerst als np.NaN kodieren


* Was machen wir danach mit Null-Werten?
 * Zeilen mit Nullwerten weglassen (wenn es nur wenige sind)
 * Spalten mit Nullwerten weglassen (wenn es zuviele sind)
 * Imputation (fehlende Werte ableiten):
   * Mittelwert, Median, häufigster Wert einer Spalte wählen
   * Wert der vorherigen oder nachfolgender Zeile verwenden (wenn es eine Reihenfolge in den Samples gibt)
   * Fehlender Wert aus anderen Werten vorhersagen, z.B. mit Regression oder Clustering
 * So lassen (einige Algorithem können damit umgehen, z.B. XGboost)
 * Die Information "Viele Nullwerte" als neuer Feature verwenden (siehe Feature Engineering weiter unten)

In [ ]:
df.isnull().any()

In [ ]:
df.dropna(inplace=True)
df.isnull().any()

###  Ausreisser behandeln

* Wie bei Null-Werten zuerst herausfinden, ob es einen legitimen Grund für Ausreisser gibt
* Ausreisser finden ist nicht ganz einfach
* Grunsätzlich zwei Ansätze:
  * Parametrisch: Anname: Daten folgen einer bestimmten Verteilung, dann ist es einfach zu definieren, wie weit weg vom Median ein Datenpunkt liegen muss, um als Ausreisser betrachtet werden zu können
  * Nicht-Parametrisch: Ohne diese Annahmem, z.B. mittels Clustering
* (Nicht entfernte) Ausreisser beinflussen die spätere Wahl des Prediction Algorithmus

### Kategorische Variablen behandeln

* Die Algorithmen brauchen numerischen Input
* Im Beispiel haben wir aber eine Spalte mit Chars


Allgemein werden bei statistischen Daten die Spalten in zwei Typen mit je zwei Untertypen eingeteilt:
* Numerisch: messbare Grösse, Zählwert
 * diskret (z.B. 1,2,3,4,5)
 * kontinuierlich (z.B. 12.152, 17.882, 20.5)
* Categorical: Ausprägung
 * nominal: ohne natürliche Ordnung, z.B. Geschlecht, Land
 * ordinal: mit Ordnung bzw. Rang, z.B. Sternebewertung bei Hotels, T-Shirt Grössen


Behandlung:
* Für ordinale Werte: enumeration
* Für nominale Werte: One-Hot (bei Trees sind enums auch OK)
* [Hashing Trick](https://en.wikipedia.org/wiki/Feature_hashing)

In [ ]:
df['haustyp (n)'].unique()

In [ ]:
df = pd.get_dummies(df, columns=['haustyp (n)'])
df.shape

In [ ]:
df.head()

## Feature engineering

* Set von Features bereitstellen
* Benötigt Domainwissen, Statistik-Knowhow und Erfahrung
* Grosser Impact auf Performance
* Neuronale Netze machen dies selber

### Feature Extraction

* Automatisches Generieren von Features aus Rohdaten
* Dimensionality reduction

### Feature construction

* Manuelles generieren mit Domain- und Statistikwissen
* Aggregieren, kombinieren, Logarithmus, Anscombe, TD-IDF usw.

### Feature selection

* Features mit viel Predictive Power auswählen
* Redundante und irrelevante Features entfernen
* Anzahl verringern

Man könnte sich zum Beispiel überlegen, dass der Preis eines Hauses in Abhängigkeit zum Renovationsjahr nicht linear zunimmt. Erst vor kurzem gemachte Renovationen bringen viel für den Preis, länger zurückliegende jedoch kaum was.

Somit könnte man aus dem Renovationsjahr eine "Zeit seit der letzten Renovation machen" und von dieser den Logarithmus nehmen.

In [ ]:
df['zeitseitrenovation'] = max(df['renovationsjahr (d)'])-df['renovationsjahr (d)']

In [ ]:
df['log_renovation (n)'] = np.log(df['zeitseitrenovation']+1)

In [ ]:
df.drop(['renovationsjahr (d)', 'zeitseitrenovation'], axis=1, inplace=True)

## Validierungsmethodik festlegen

#### Cross-Validation

* Um einen Algorithmus zu trainieren und anschliessend zu validieren, werden die Trainingsdaten aufgeteilt
* Einfache Stragegie: Zwei Teile, ca 80% zum Trainieren und 20% zum Validieren
* Wie die Daten aufgeteilt werden, ist nicht immer offensichtlich (shuffle, strafification, time series)


* Achtung bei der Begrifflichkeit:
  * Meist wird in Beispielen das mit Labels versehene Trainingsset in *train* und *test* aufgeteilt, und test zum validieren verwendet
  * Manchmal (und das ist eigentlich die bessere Terminologie) wird das mit Labels versehene Trainingsset in *train* und *validation* aufgeteilt, während man davon ausgeht, dass für das Testset keine Labels vorhanden sind.

In [ ]:
df.shape

Wir teilen unser Trainingsset in zwei Teile, 80% davon verwenden wir für das Training unseres Algorithmus und 20% legen wir zur Seite, um nach erfolgtem Training überprüfen zu können, wie unsere trainierten Algorithmen abschneiden.

Zudem lagern wir das Target (den Preis) in eine eigene Variable aus. Dies, weil die von uns verwendete Machine Learning Bibliothek [Scikit-Learn (sklearn)](http://scikit-learn.org/) das so vorsieht.

In [ ]:
y = df['PREIS']
df.drop(['PREIS'], axis=1, inplace=True)
df.head()

Nun verwenden wir zum ersten Mal in diesem Beispiel die Python Machine Learning Bilbiothek [Scikit-Learn](http://scikit-learn.org/stable). Wir verwenden die Funktion train_test_split auf unseren Dataframe mit den Trainingsdaten und der Variable y mit der Targetvariable, und erhalten 4 Rückgabewerte:
 * X_train (Features zum Trainieren)
 * y_train (Target zum Trainieren, wir machen hier ja SuperVised Learning)
 * X_test (Features zum validieren, legen wir zur Seite)
 * y_test (Target zum validieren, legen wir zur Seite)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=1)

In [ ]:
(df.shape, X_train.shape, X_test.shape)

## Training (Model building)

Nun folgt ein iterativer Schritt, bei dem verschiedene Algorithmen mit verschiedenen Parametern trainiert und die Resultate verglichen werden.

Natürlich ist immer auch der gesamte Prozess iterativ, man geht immer wieder auch zurück zur Explorativen Datenanalyse und zum Feature Engineering.

### Algorithmus trainieren

Wir wählen im Beispiel nun zwei Algorithmen. Einmal Linear least squares (Lineare Regression) mit L2 Regularization und einmal Random Forests.

In [ ]:
from sklearn import linear_model

lr = linear_model.Ridge(alpha=.5, normalize=True)
lr.fit(X_train, y_train) # lr ist nun unser trainiertes Model (Linear Regression)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=72)
rf.fit(X_train, y_train) # rf ist nun unser trainiertes Model (Random Forests)

### Hyperparameter optimieren

Als Hyperparameter werden diejenigen Parameter bezeichnet, welche beeinflussen, wie der Algorithmus sich beim Training verhält. Die "nicht-hyper-parameter" sind diejenigen, die der Algorithmus durch das Training ermittelt (also die Parameter des trainierten Modells).

Im obigen Beispiel sind dies alpha, normalize und n_estimators (n_jobs gibt lediglich die Anzahl der zu verwendenden CPU Cores an, random_state seedet den Zufallszahlengenerator).

Hyperparameter haben einen grossen Einfluss auf die Performanz eines Algorithmus.

### Validieren

Haben wir einen Algorithmus trainiert, möchten wir nun wissen, wie gut er in Bezug auf unsere oben definierte Metrik, dem Root Mean Squared Error $\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}$ abschneidet.

Zuerst verwenden wir nun unsere beiden trainierten Modelle, und machen Predictions für das Test Set (die 20%, welche wir oben zur Seite gelegt haben).

In [ ]:
pred_lr = lr.predict(X_test) # Linear Regression
pred_rf = rf.predict(X_test) # Random Forests

pred_lr und pred_rf enthalten nun die Immobilienpreise, die unser Algorithmus für die 20% des zur Seite gelegten Testsets voraussagt. Wir vergleichen diese Voraussagen nun mit den tatsächlichen Verkaufspreisen. Als Mass verwenden wir unseren RMSE.

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
# Linear Regression
rmse_lr = np.sqrt(mean_squared_error(y_test, pred_lr))
rmse_lr

In [ ]:
# Random Forests
rmse_rf = np.sqrt(mean_squared_error(y_test, pred_rf))
rmse_rf

Mit dem RMSE können wir unsere zwei Modelle vergleichen, und sehen, dass Random Forests besser als Lineare Regression ist (wir messen den Error, je kleiner der Wert desto besser also).

#### Einschub

#### Cross-Validation

* Eine etwas aufwändigere Strategie zum Validieren: k-fold Cross Validation, dabei werden
  * die Trainingsdaten in k Teile aufgeteilt,
  * jeder Teil ist wird einmal zum Validieren verwendet, während die anderen k-1 Teile zum Trainieren verwendet werden
  * anschliessend wird der Mittelwert berechnet
  * k liegt meist zwischen 3 und 10

Wie stark unsere (zufällige) Aufteilung in 80% für das Training und 20% für die Validierung das Validierungsresultat beeinflusst, sehen mir mit k-Fold Cross Validation. Wir führen unsere Aufteilung in 80/20 fünf mal durch, so dass jedes Sample einmal im Validierungsset ist.

In [ ]:
# 5 folds
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, pd.concat([X_train, X_test]),
                         pd.concat([y_train, y_test]),
                         cv=5, scoring='neg_mean_squared_error')
np.sqrt(-1*scores)

Wie wir sehen, hat die Aufteilung in unserem Beispiel einen starken Einfluss auf unseren RMSE. 

## Ensembling

Unter ensembling verseht man das Kombinieren mehrerer Algorithmen, um das Resultat zu verbessern. Wir kombinieren im Folgenden die Resultate unserer zwei Algorithmen, indem wir den gewichteten Durchschnitt beider Resultate nehmen (da Random Forests etwas besser abgeschnitten hat, geben wir ihm etwas mehr Gewicht).

Es gibt viele weitere und komplexere Möglichkeiten, verschiedene Algorithmen zu kombinieren.

In [ ]:
combined_error = rmse_lr + rmse_rf
weight_lr = 1-rmse_lr/combined_error
weight_rf = 1-rmse_rf/combined_error

print("Lineare Regression:\t {}".format(rmse_lr))
print("Random Forests:\t\t {}".format(rmse_rf))
print("Weighted Avgerage:\t {}".format(np.sqrt(mean_squared_error(y_test, weight_lr*pred_lr + weight_rf*pred_rf))))

OK, das bringt nicht immer was...

# Prediction

Nun kann man das Modell bzw. den trainierten Algorithmus auf unbekannte Daten anwenden. Wir haben im Beispiel zwar keine solchen, tun aber so, als ob:

In [ ]:
predictions = rf.predict(X_test)
predictions[0:10]